In [209]:
import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [210]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [211]:
"""
legit = pd.read_csv("/content/drive/MyDrive/DA20 - Final Project/Supervised Learning/legitimate_urls.csv")
phishing = pd.read_csv("/content/drive/MyDrive/DA20 - Final Project/Supervised Learning/phishing_urls.csv")


# add a 'status' column to the dataframes
legit["status"] = "legitimate"
phishing["status"] = "phishing"

# combine the two dataframes
df = pd.concat([legit, phishing], ignore_index=True)

# shuffle the dataframe's rows randomly
df = df.sample(frac=1, random_state=42) # Set random_state to an integer for reproducibility

df
"""

'\nlegit = pd.read_csv("/content/drive/MyDrive/DA20 - Final Project/Supervised Learning/legitimate_urls.csv")\nphishing = pd.read_csv("/content/drive/MyDrive/DA20 - Final Project/Supervised Learning/phishing_urls.csv")\n\n\n# add a \'status\' column to the dataframes\nlegit["status"] = "legitimate"\nphishing["status"] = "phishing"\n\n# combine the two dataframes\ndf = pd.concat([legit, phishing], ignore_index=True)\n\n# shuffle the dataframe\'s rows randomly\ndf = df.sample(frac=1, random_state=42) # Set random_state to an integer for reproducibility\n\ndf\n'

In [212]:
df = pd.read_csv("/content/drive/MyDrive/DA20 - Final Project/Supervised Learning/urls.csv")

df.head(5)

,url,status
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [213]:
df.loc[df["status"] == "benign", "status"] = 'legitimate'
df.loc[df["status"] == "malware", "status"] = 'phishing'
df.loc[df["status"] == "defacement", "status"] = 'phishing'

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   status  651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [215]:
df.isnull().sum()
df.dropna(inplace = True)
df

,url,status
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,legitimate
2,bopsecrets.org/rexroth/cr/1.htm,legitimate
3,http://www.garage-pirenne.be/index.php?option=...,phishing
4,http://adventure-nicaragua.net/index.php?optio...,phishing
...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing
651188,www.gamespot.com/xbox360/action/deadspace/,phishing
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing


In [216]:
# Data selection
feature_df = df['url']
status_df = df['status']

# Transform data
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# Vectorize URLs
vectorizer = CountVectorizer()
feature_vec = vectorizer.fit_transform(feature_df)



# Encode status column
label_encoder = LabelEncoder()
status_arr = label_encoder.fit_transform(status_df)

label_encoder = LabelEncoder()
status_arr = label_encoder.fit_transform(status_df)

# Get class labels and corresponding encoded values
class_labels = label_encoder.classes_
class_encodings = label_encoder.transform(class_labels)

# Print class labels and their corresponding encoded values
for label, encoding in zip(class_labels, class_encodings):
    print(f"{encoding} -> {label}")


0 -> legitimate
1 -> phishing


In [217]:
#chia tap train, test
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature_vec, status_arr, test_size=0.25, random_state=42)

print('Train: ', x_train.shape[0])
print('Test: ', x_test.shape[0])

Train:  488393
Test:  162798


In [218]:
#modeling lbfgs
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='lbfgs',C= 10, class_weight= 'balanced', max_iter= 500, penalty= 'l2')
lr_lbfgs = logreg.fit(x_train,y_train)


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
"""
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10],
              'penalty': ['l1', 'l2'],
              'max_iter': [100, 500, 1000],
              'class_weight': [None, 'balanced']}

grid_search = GridSearchCV(LogisticRegression(solver='lbfgs'),
                           param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)

grid_search.fit(x_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")


# Output:
# Best parameters: {'C': 10, 'class_weight': 'balanced', 'max_iter': 500, 'penalty': 'l2'}
# Best cross-validation score: 0.9602328470257125
"""

In [ ]:
y_pred=logreg.predict(x_test)
print(y_pred)

In [ ]:
from sklearn.metrics import jaccard_score
print(jaccard_score(y_test, y_pred,pos_label=0))

In [ ]:
logreg.score(x_test, y_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 score: {f1:.2f}")


In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
class_names=[] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
"""
new_url = input('Enter an URL to test: ')

# Transform features into same format as training data
new_vector = vectorizer.transform([new_url])

# Make prediction using trained model
prediction = logreg.predict(new_vector)

if prediction[0] == 0:
  print('Legitimate')
else:
  print('Phishing')
"""

## Test

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/DA20 - Final Project/Supervised Learning/publish_test.csv')
test_df.head()

In [ ]:
test = test_df[['x','y']]

test.isnull().sum()
test.dropna()

test.info()

In [ ]:
indexValue = test[test['y'] == 'continue=to&amp'].index
test.drop(indexValue , inplace=True)

In [ ]:
x_t = test['x']
y_t = test['y'].astype(int)


x_t = vectorizer.transform(x_t)
y_t = np.asarray(y_t)
print(y_t)

y_pred_t = logreg.predict(x_t)
print(y_pred_t)

In [ ]:
logreg.score(x_t, y_t)

In [ ]:
from sklearn.metrics import jaccard_score
print(jaccard_score(y_t, y_pred_t,pos_label=0))

In [ ]:
accuracy = accuracy_score(y_t, y_pred_t)
f1 = f1_score(y_t, y_pred_t)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 score: {f1:.2f}")